# Проект по анализу ЗП.

## Описание данных:

Для анализа предоставлены данные о видах, стоимости каждой услуги одного из сотрудников Graph Design компании.


**Виды услуг:**
 - Создание сайта,
 - Логотип, 
 - Инстаграм,
 - Фейсбук, 
 - Вконтакте,
 - Чат-бот,
 - Баннер,
 - Визитка

**Стоимость проектов:**
 - Создание сайта - 1600 руб.
 - Логотип - 13831 руб.
 - Инстаграм - 7990 руб.
 - Фейсбук - 5320 руб.
 - Вконтакте - 7990 руб.
 - Чат-бот - 7820 руб.
 - Баннер - 2000 руб.
 - Визитка - 2000 руб.


**Названия столбцов:**


 - created - Клиент сделал заказ.
 - start_date - Проект взят в работу.
 - end_date	- Проект отдан клиенту.
 - projects - Название проекта.	

**Цель** проекта - проанализировать динамику заработной платы сотрудника, выявить определенные закономерности и тенденции, проиллюстрировать результаты с помощью инструментов визуализации.

## Обработка данных

In [1]:
import pandas as pd
import locale

import datetime  
from datetime import datetime

from dateutil.relativedelta import relativedelta
from datetime import date

from plotly import graph_objects as go
import matplotlib.pyplot as plt

import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

from IPython.display import display, HTML
from scipy import stats as st

import numpy as np
import statsmodels.api as sm

from statsmodels.formula.api import ols
import plotly.io as pio

In [ ]:
data = pd.read_csv('design_data.csv')

In [ ]:
def first_look (df):
  '''Функция получения первичной информации о датафрейме'''
  print ('------------- Первые 5 строк ------------')
  display(df.head())
  print('')
  print('')
  print ('------------- Типы данных ------------')
  print (df.info())
  print('')
  print('')
  print ('------------- Пропуски ------------')
  count = 0
  for element in df.columns:
    if df[element].isna().sum() > 0: 
      print(element, ' - ', df[element].isna().sum(), 'пропусков')
      count = +1
  if count == 0:
    print('Пропусков НЕТ')
  print('')
  print('')
  print ('------------- Дубликаты ------------')
  if df.duplicated().sum() > 0:
    print('Дубликатов: ', df.duplicated().sum())
  else:
    print('Дубликатов НЕТ')

In [ ]:
first_look(data)

По первой оценке данных видно, что столбцы с датами не приведены к нужному типу, так же как и названия столбцов.

In [ ]:
data.columns = data.columns.str.lower()

In [ ]:
data.head()

In [ ]:
#Уберем предлог "в" и время, чтобы привести всю дату к единому формату.
data['created'] = data['created'].str.split(' в').str[0]

In [ ]:
locale.setlocale(
    category=locale.LC_ALL,
    locale="Russian"
)

data['start_date'] = pd.to_datetime(data['start_date'], format = '%d %b %Y')
data['end_date'] = pd.to_datetime(data['end_date'], format = '%d %b %Y')
data['created'] = pd.to_datetime(data['end_date'], format = '%d %b %Y')

In [ ]:
data['created'].min()

In [ ]:
data['end_date'].max()

In [ ]:
days = data['end_date'].max()-data['created'].min()
days = int(str(days).split(' ')[0])
days

In [ ]:
week = 7

In [ ]:
weeks = days/week
weeks

In [ ]:
print(f'Данные для анализа находятся в промежутке с {data.created.min().strftime("%Y-%m-%d")} по {data.end_date.max().strftime("%Y-%m-%d")}, то есть {days} день.')

In [ ]:
project = data.projects.value_counts()
project

In [ ]:
print(f'Больше всего было выполнено проектов по проектам {project.index[0]}, а также {project.index[1]} и {project.index[2]}. Неприоритетными в этом отчетном периоде оказались: {project.index[-1]}, {project.index[-2]} и {project.index[-3]}.')

Обратим внимание на стоимость проектов :

 - Логотип - 13831 руб.
 - Инстаграм - 7990 руб.
 - Вконтакте - 7990 руб.
 - Чат-бот - 7820 руб.
 - Фейсбук - 5320 руб.
 - Баннер - 2000 руб.
 - Визитка - 2000 руб.
 - Создание сайта - 1600 руб.

Для максимизации прибыли необходимо выполнять проекты из первой 3-ки или 5-ки из списка стоимотей проектов. В 3-ке наиболее часто выполняемых проектов: сам проект - самый прибыльный, а также декомпозиция и Выпускной проект, которые соответствуют минимальной и средней стоимости соответственно. 

С точки зрения максимизации прибыли - ситуация может быть улучшена засчет решения проектов с большей стоимостью, однако на текущей вакансии присутствуют другие обязательные параметры выбора проектов, на которые сотрудник не может повлиять.

## Расчеты

Напишем функцию для подсчета ЗП за текущий период.

In [ ]:
prices = {
    'создание_сайта': 1600,
    'логотип': 13830,
    'inst': 7990,
    'facebook': 5320,
    'VK': 7990,
    'чат-бот': 7820,
    'баннер': 2000,
    'визитка': 2000
}

In [ ]:
data['income'] = data['projects'].map(prices)

In [ ]:
salary = data['income'].sum()

In [ ]:
print(f' Зарпалата за текущий период без учета налога = {salary}, ЗП с учетом налога = {round(salary * 0.94,2)} ')

In [ ]:
sal_week = round(salary/weeks)
print(f'Средняя зарплата за неделю составляет {sal_week} рублей')

In [ ]:
sal_day = salary/days
print(f'Средняя зарплата за день составляет {round(sal_day,2)} рублей')

In [ ]:
print(f'ЗП за {days} день составила - {salary} руб. без учета налога. С учетом налога: {round(salary * 0.94,2)}. В среднем получается, что недельный заработок составил {sal_week} руб, а дневной - {sal_day} руб.')

In [ ]:
all_pr = len(data.projects)
print(f' Всего за {days} день было сделано {all_pr} проектов')

In [ ]:
pr_week = all_pr/3
print(f' В среднем за неделю выполняется {round(pr_week,2)} проектов')

In [ ]:
pr_day = all_pr/21
print(f' В среднем за день выполняется {round(pr_day,2)} проектов')

In [ ]:
print(f' В среднем в день 1 проект приносит {round(sal_day/pr_day, 2)} рублей')

In [ ]:
accepted_pr = data.groupby('start_date')['projects'].count().sort_values(ascending = False)
accepted_pr

In [ ]:
print(f'Больше всего проектов было принято {accepted_pr.index.strftime("%Y-%m-%d")[0]}, а также {accepted_pr.index.strftime("%Y-%m-%d")[1]} и {accepted_pr.index.strftime("%Y-%m-%d")[2]} от {accepted_pr.values[0]} до {accepted_pr.values[2]} соответсвенно. Наименее активными оказались {accepted_pr.index.strftime("%Y-%m-%d")[-1]} и {accepted_pr.index.strftime("%Y-%m-%d")[-2]}, менее {accepted_pr.values[-2]} проектов  ')

In [ ]:
completed_pr = data.groupby('end_date')['projects'].count().sort_values(ascending = False)
completed_pr

In [ ]:
print(f'Завершено проектов больше всего было {completed_pr.index.strftime("%Y-%m-%d")[0]} - {completed_pr.values[0]} проектов, а также {completed_pr.index.strftime("%Y-%m-%d")[-1]}. Меньше всего проектов завершилось {completed_pr.index.strftime("%Y-%m-%d")[-1]},{completed_pr.index.strftime("%Y-%m-%d")[-2]} и {completed_pr.index.strftime("%Y-%m-%d")[-3]}.')

In [ ]:
clients = data.groupby('created')['projects'].count().sort_values(ascending = False)
clients

In [ ]:
print(f'Клиенты тоже были активными {clients.index.strftime("%Y-%m-%d")[0]} и прислали целых {clients.values[0]} проектов за день, {clients.index.strftime("%Y-%m-%d")[1]} тоже был пик, но уже поменьше - {clients.values[1]} проектов. Меньше всего клиенты заказывали проекты в {clients.index.strftime("%Y-%m-%d")[-1]},{clients.index.strftime("%Y-%m-%d")[-2]} и {clients.index.strftime("%Y-%m-%d")[-3]}  числах.')

In [ ]:
print(f'Проанализировав активность отдельно по датам, можно сделать вывод, что дизайнер берет проект ,чаще всего, в тот же день, в какой его присылает заказчик. Определенный пик можно наблюдать {clients.index.strftime("%Y-%m-%d")[0]} и {clients.index.strftime("%Y-%m-%d")[1]}. Интересно, связано ли эта активность с днем недели.')

In [ ]:
data['week_day_started'] = data['start_date'].dt.day_name()
data['week_day_created'] = data['created'].dt.day_name()
data['week_day_ended'] = data['end_date'].dt.day_name()

Добавив колонку с днем недели для даты, когда клиент прислал проект, и для даты, когда дизайнер принял проект, теперь можно определить, в какой день недели наибольшая активность.

In [ ]:
data.head()

In [ ]:
peak_days = data[data.start_date.isin([clients.index[0], clients.index[1]])]
peak_days.head()

In [ ]:
print(f'Мы видим, что пиковые дни по кол-ву проектов приходятся на {clients.index.strftime("%Y-%m-%d")[0]}, то есть {peak_days.iat[0,4]}, и {clients.index.strftime("%Y-%m-%d")[1]} - {peak_days.iat[-1,4]}.')

In [ ]:
count_of_pr_per_wd = data.groupby('week_day_started')['projects'].count().sort_values(ascending = False)
count_of_pr_per_wd

In [ ]:
print(f'{count_of_pr_per_wd.index[0]} - тяжелый день, на него приходится наибольшее кол-во работы. А меньше всего проектов ревьюер проверяет в {count_of_pr_per_wd.index[-1]} и в {count_of_pr_per_wd.index[-2]}.')

In [ ]:
dyn_by_clients = data.groupby('week_day_created')['projects'].count().sort_values(ascending = False)
dyn_by_clients

In [ ]:
print(f'А вот клиенты имеют другую динамику по заказу проектов. Явными лидерами стали - {dyn_by_clients.index[0], dyn_by_clients.index[1]}. Меньше всего проектов присылают в {dyn_by_clients.index[-1]}.')

## Визуализация

**Дашборд по динамике количества выполненных проектов в разрезе дней**

In [ ]:
dynamics = data.groupby('week_day_ended')['projects'].count().sort_values(ascending = False )
dynamics

In [ ]:
pio.renderers.default = "svg"
svg_renderer = pio.renderers["svg"]
svg_renderer.scale = 1.2

In [ ]:
fig_ = px.bar(dynamics, x=dynamics.index, y=dynamics, title='Динамика кол-ва выполненных проектов по дням недели', labels=dict(week_day_ended = 'день недели', y='кол-во выполненных проектов'))
fig_.update_xaxes(tickangle=45) # повернём подписи по оси X на 50 градусов
fig_.show("svg")

In [ ]:
print(f'Динамика выполнения проектов говорит о том, что пик завершенных проектов приходится на {dynamics.index[0]} и {dynamics.index[1]} - {dynamics.values[0]} и {dynamics.values[1]} проектов соответсвенно, к {dynamics.index[-0]} мы видим минимум выполненных проектов в статусе "завершено" - {dynamics.values[-1]} проектов, почти в {round(dynamics.values[0]/dynamics.values[-1],2)} раза меньше, чем пиковое значение. Следовательно можно выделить самые продуктивные дни - {dynamics.index[0]}, {dynamics.index[1]}. {dynamics.index[2]} и {dynamics.index[3]} тоже близки к пику - {dynamics.values[2]} и {dynamics.values[3]} проектов.')

In [ ]:
dyn_per_days = data.groupby('end_date')['projects'].count().sort_values(ascending = False)
dyn_per_days

In [ ]:
fig = px.bar(dyn_per_days, x=dyn_per_days.index, y=dyn_per_days, title='Динамика кол-ва выполненных проектов датам', labels=dict(end_date = 'дата', y='кол-во выполненных проектов'))
fig.update_xaxes(tickangle=50) # повенём подписи по оси X на 50 градусов
fig.update_xaxes(tickangle=50, tickmode='linear', dtick='D1')
fig.add_hline(y=dyn_per_days.values.mean(), line_width=3, line_dash="dash", line_color="red")
fig.show("svg")

Среднее кол-во проектов за рассматриваемый период составляет около 8-ми проектов. То есть , переходя за отметку 8, сотрудник делает сверхнормы. Таких дней на графике мы видим - 9 из 21.
  

In [ ]:
print(f'Пиковое значение по количеству выполненных проектов приходится на {dyn_per_days.index.strftime("%Y-%m-%d")[0]}.Меньше всего проектов было сделано {dyn_per_days.index.strftime("%Y-%m-%d")[-1]}. Размах составляет {round(dyn_per_days.values[0]/dyn_per_days.values[-1],2)} проектов. В среднем в день было выполнено {round(dyn_per_days.values.mean(),2)} проектов. ')

In [ ]:
data['month'] = data['end_date'].dt.month

In [ ]:
data.head()

In [ ]:
months = data.groupby('month')['projects'].count().sort_values(ascending = False )
months
print(f'за {months.index[0]}-й месяц было выполнено - {months.values[0]} проектов, а за  {months.index[1]}-й было выполнено {months.values[1]} проектов')

In [ ]:
fig = go.Figure(data=[go.Pie(labels=months.index, values=months)])
fig.update_layout(title_text='Кол-во выполненных проектов по месяцам', title_x= 0.5)
fig.show("svg") 

**Дашборд по заработанной сумме в разрезе дней**

In [ ]:
profit = data.groupby('projects')['income'].agg(['count','sum'])
profit['%_of_salary'] = (profit['sum']/salary)*100
profit = profit.sort_values('sum', ascending = False)
profit

In [ ]:
fig3 = px.bar(profit, x=profit.index, y=profit['sum'], title='Динамика ЗП по проектам', labels=dict(home_task = 'название проекта', sum='ЗП'))
fig3.update_xaxes(tickangle=50) # повернём подписи по оси X на 50 градусов
fig3.add_hline(y=profit['sum'].mean(), line_width=3, line_dash="dash", line_color="red")
fig3.show("svg")

In [ ]:
fig = go.Figure(data=[go.Pie(labels=profit.index, values=profit['%_of_salary'])])
fig.update_layout(title_text='Распределение ЗП по проектам', title_x= 0.5)
fig.show("svg") 

In [ ]:
print(f'Наибольший доход принес заказ  {profit.index[0]} - {profit.iat[0,1]} р., что составило почти  {round(profit.iat[0,2],2)}% от общей выручки. На втором месте по доходности оказался проект {profit.index[1]} - {profit.iat[1,1]} р., что составило {round(profit.iat[1,2],2)}% от всей зп. Наименее доходными оказались проекты : {profit.index[-1]} и {profit.index[-2]}, каждый из которых принес по {profit.iat[-1,1]} и {profit.iat[-2,1]} р. соответственно')

In [ ]:
income_per_d = data.groupby('end_date')['income'].sum().sort_values(ascending = False)
income_per_d

In [ ]:
fig2 = px.bar(income_per_d, x=income_per_d.index, y=income_per_d, title='Динамика по заработанной сумме в разрезе дней', labels=dict(end_date = 'дата', y='доход'))
fig2.update_xaxes(tickangle=45) 
fig2.add_hline(y=income_per_d.values.mean(), line_width=3, line_dash="dash", line_color="red")
fig2.update_xaxes(tickangle=50, tickmode='linear', dtick='D1')
fig2.show("svg")

Говоря о прогнозах на будущее, на основе текущих данных, можно сказать, что постепенно уровень ЗП будет выравниваться до среднего уровня, будет меньше отклонений\аномальных значений, выше или ниже нормы. Мы по-прежнему будем наблюдать пиковые значения, так как в определенные дни скапливаются все проекты, которые нужно закрыть, и именно за этот день считается прибыль.

In [ ]:
print(f'В разрезе дней {income_per_d.index.strftime("%m-%d")[0]} и {income_per_d.index.strftime("%m-%d")[1]} - самые прибыльные дни {income_per_d.values[0]} р. и {income_per_d.values[1]} р. соответственно. Меньше всего дохода пришло {income_per_d.index.strftime("%m-%d")[-1]} и {income_per_d.index.strftime("%m-%d")[-2]}, {income_per_d.values[-1]} и {income_per_d.values[-2]} рублей соответственно.')

In [ ]:
print(f'Средний доход в день составляет примерно {round(income_per_d.values.mean(),2)} рублей, следовательно выше этой суммы сотрудник делает работу сверхнормы')

In [ ]:
income_per_wd = data.groupby('week_day_ended')['income'].sum().sort_values(ascending = False )
income_per_wd

In [ ]:
fig4 = px.bar(income_per_wd, x=income_per_wd.index, y=income_per_wd, title='Динамика по заработанной сумме в разрезе дней недели', labels=dict(week_day_ended = 'дни недели', y='доход'))
fig4.update_xaxes(tickangle=45) 
fig4.add_hline(y=income_per_wd.values.mean(), line_width=3, line_dash="dash", line_color="red")
fig4.show("svg")

In [ ]:
print(f'{income_per_wd.index[0]} самый доходный день - {income_per_wd.values[0]} руб. было заработано в эти дни, а {income_per_wd.index[-1]} и {income_per_wd.index[-2]} наименее прибыльны - {income_per_wd.values[-1]} и {income_per_wd.values[-2]} рублей соответсвенно было получены в каждый из дней. В среднем в эти дни в {round(income_per_wd.values[0]/((income_per_wd.values[-2]+income_per_wd.values[-1])/2),2)} раза меньше было заработка, чем в {income_per_wd.index[0]}')

## Дашборды в Tableau

Динамика прибыли и кол-ва проектов в разрезе дней

https://public.tableau.com/app/profile/.25533259/viz/1_16907949848390/Dashboard1?publish=yes

Динамика по доходу и кол-ву выполненных проектов в разрезе дат и вида проектов

https://public.tableau.com/app/profile/.25533259/viz/2_16907950449170/Dashboard2?publish=yes

## Статистика

**Проверка гипотез с помощью т-теста**

За представленный период воскресенья и среды принесли похожий доход: 22 и 19,9 тысяч соответсвенно. Рассмотрим существующие различия с точки зрения статистики.

Для проверки гипотез используем т-критерий, так как именно он помогает сравнить две выборки между собой на предмет статистически значимых различий. Из существующих уровней значимости выберем 0,05, то есть существует 5% того, что результат обусловлен случайностью.

Для использования т-теста необходимы 2 условия:

 - гомогенность дисперсий
 
 - Нормальность распределения в группах 

Проверим для начала распределение в группах на **нормальность:**

In [ ]:
wed = data[data['week_day_ended']=='Wednesday']['income']
sun = data[data['week_day_ended']=='Sunday']['income']

In [ ]:
wednesday = wed.values

# Построение QQ-plot
st.probplot(wednesday, dist="norm", plot=plt)

plt.xlabel('Теоретические квантили')
plt.ylabel('Наблюдаемые квантили')
plt.title('QQ-plot')


plt.show()

In [ ]:
sunday = sun.values

# Построение QQ-plot
st.probplot(sunday, dist="norm", plot=plt)

plt.xlabel('Теоретические квантили')
plt.ylabel('Наблюдаемые квантили')
plt.title('QQ-plot')


plt.show()

Нам известно, что графики построены по ценам на проекты, поэтому данные распределены ценовыми сегментами в районе 250 , 750 и 1300. Учитывая специфику данных, отклонение от диагонали не так критичны. Следовательно, мы можем использовать данные для проведения т-теста.

Проверим данные на **гомогенность дисперсий** с помощью теста Левена:

In [ ]:
st.levene(wednesday, sunday, center='median')

P - value имеет показатель > 0,05. Следовательно, мы не отвергаем гипотезу о равенстве дисперсий. Таким образом, параметр о гомогенности дисперсий соблден. -> Можем проводить т-тест.

**Нулевая гипотеза:** Статистически значимые различия между доходом от сред и вскр-й отсутствуют.
    
**Альтернативная гипотеза:** Статистически значимые различия между доходом от сред и от вскр-й существуют.

In [ ]:
# Первая гипотеза

alpha = .05

results = st.ttest_ind(
    wed, 
    sun)

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Т-криетрий Стьюдента показал, что статистически значимых различий между доходом от сред и воскресений нет.

Рассмотрим еще две выборки. Доход за 22 июня и за 5 июля - за самые прибыльные дни. Есть ли между ними статистически значимые различия ?

Для проверки гипотез также используем т-критерий с аналогичным уровнем значимости и проверим аналогичным образом условия для проведения т-теста.

In [ ]:
june = data[data['end_date']=='2023-06-22']['income']
july = data[data['end_date']=='2023-07-05']['income']

In [ ]:
jun = june.values

# Построение QQ-plot
st.probplot(jun, dist="norm", plot=plt)

plt.xlabel('Теоретические квантили')
plt.ylabel('Наблюдаемые квантили')
plt.title('QQ-plot')


plt.show()

In [ ]:
jul = july.values

# Построение QQ-plot
st.probplot(jul, dist="norm", plot=plt)

plt.xlabel('Теоретические квантили')
plt.ylabel('Наблюдаемые квантили')
plt.title('QQ-plot')


plt.show()

In [ ]:
st.levene(jun, jul, center='median')

Мы видим аналогичную ситуацию с нормальным распределением и тестом Левена.-> Также можем провести т-тест.

**Нулевая гипотеза:** Статистически значимые различия между доходом от 22 июня и от 5 июля отсутствуют.
    
**Альтернативная гипотеза:** Статистически значимые различия между доходом от 22 июня и от 5 июля существуют.

In [ ]:
# Втрая гипотеза

results = st.ttest_ind(
    june, 
    july)

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Т-критерий Стьюдента показал, что статистически значимых различий между доходом от самых прибыльных дней - нет, то есть между 5 июля и 22 июня.

**Проведение двустороннего дисперсионного анализа0**

Проведем двусторонний ANOVA, возьмем доход за зависимую переменную и посмотрим, как на нее влияет месяц и день недели.

In [ ]:
# Проведение двустороннего дисперсионного анализа
model = ols('income ~ C(week_day_ended) + C(month)', data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)

Интерпретация результатов :

 - Последний столбец показывает p- уровень значимости. День недели p-value < 0,05. Следовательно, фактор дня недели оказывает статистически значимый результат. 
 - А вот p- уровень значимости для месяца >0,05. Получается, что фактор месяца в данном случае никак не влияет на доход.

In [ ]:
display(HTML("<h1>Выводы:</h1>"))
print()
print(f' - Данные для анализа находятся в промежутке с {data.created.min().strftime("%Y-%m-%d")} по {data.end_date.max().strftime("%Y-%m-%d")}, то есть \033[1m{days}\033[0m день.')
print()
print(f' - {income_per_wd.index[0]} самый \033[1mдоходный\033[0m день - {income_per_wd.values[0]} руб. было заработано в эти дни, а {income_per_wd.index[-1]} и {income_per_wd.index[-2]} \033[1mнаименее прибыльны\033[0m - {income_per_wd.values[-1]} и {income_per_wd.values[-2]} рублей соответсвенно было получены в каждый из дней.')
print()
print(f' - В разрезе дней {income_per_d.index.strftime("%m-%d")[0]} и {income_per_d.index.strftime("%m-%d")[1]} - самые \033[1mприбыльные дни\033[0m {income_per_d.values[0]} р. и {income_per_d.values[1]} р. соответсвенно. \033[1mМеньше всего дохода\033[0m пришло {income_per_d.index.strftime("%m-%d")[-1]} и {income_per_d.index.strftime("%m-%d")[-2]}, {income_per_d.values[-1]} и {income_per_d.values[-2]} рублей соответственно.')
print()
print(f' - \033[1mНаибольший доход\033[0m принес \033[1mпроект\033[0m {profit.index[0]} - {profit.iat[0,1]} р., что составило почти  {round(profit.iat[0,2],2)}% от общей выручки. На втором месте по доходности оказался проект {profit.index[1]} - {profit.iat[1,1]} р., что составило {round(profit.iat[1,2],2)}% от всей зп. \033[1mНаименее доходными\033[0m оказались проекты : {profit.index[-1]} и {profit.index[-2]}.')
print()
print(f' - за {months.index[0]}-й месяц было выполнено - {months.values[0]} проектов, а за  {months.index[1]}-й было выполнено {months.values[1]} проектов')
print()  
print(f' - \033[1mПиковое значение\033[0m по количеству выполненных \033[1mпроектов\033[0m приходится на {dyn_per_days.index.strftime("%Y-%m-%d")[0]}.\033[1mМеньше всего проектов\033[0m было сделано {dyn_per_days.index.strftime("%Y-%m-%d")[-1]}. Размах составляет {round(dyn_per_days.values[0]/dyn_per_days.values[-1],2)} проектов. В среднем в день было выполнено {round(dyn_per_days.values.mean(),2)} проектов. ')
print()
print(f' - {count_of_pr_per_wd.index[0]} - тяжелый день, на него приходится наибольшее кол-во работы. А меньше всего проектов ревьюер проверяет в {count_of_pr_per_wd.index[-1]} и в {count_of_pr_per_wd.index[-2]}.')
print()
print(f' - Мы видим, что \033[1mпиковые дни\033[0m по кол-ву проектов приходятся на {clients.index.strftime("%Y-%m-%d")[0]}, то есть {peak_days.iat[0,4]}, и {clients.index.strftime("%Y-%m-%d")[1]} - {peak_days.iat[-1,4]}.')
print()
print(f' -  В среднем \033[1mв день\033[0m 1 проект приносит {round(sal_day/pr_day, 2)} рублей')
print()
print(f' -  В среднем \033[1mза день\033[0m выполняется {round(pr_day,2)} проектов')
print()
print(f' -  В среднем \033[1mза неделю\033[0m выполняется {round(pr_week,2)} проектов')
print()
print(f' - \033[1mЗП\033[0m за {days} день составила - {salary} руб. без учета налога. \033[1mС учетом налога\033[0m: {round(salary * 0.94,2)}. В среднем получается, что \033[1mнедельный заработок\033[0m составил {sal_week} руб, а \033[1mдневной\033[0m - {sal_day} руб.')
print()
print(f' - \033[1mБольше всего выполнено проектов\033[0m таких как {project.index[0]}, а также {project.index[1]} и {project.index[2]}. Неприоритетными в этом отчетном периоде оказались: {project.index[-1]}, {project.index[-2]} и {project.index[-3]}.')